In [2]:
from os_helper import OsHelper
import pandas as pd

In [ ]:
from compute_metrics import ComputeMetrics

os_helper = OsHelper()

price_data = os_helper.read_data(directory_name="base data", file_name="close_fixed.csv", index_col=0, low_memory=False)
trade_volume = os_helper.read_data(directory_name="base data", file_name="volume_fixed.csv", index_col=0, low_memory=False)
spy = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0, low_memory=False)

print(spy)

rolling_window_years = [1, 2, 3]

# Compute all risk metrics needed for the strategy and the filter
metrics_calculator = ComputeMetrics(price_data=price_data, trade_volume=trade_volume, rolling_window_years=rolling_window_years,
                                    benchmark_series=spy['Adj Close'])

final_df = metrics_calculator.compile_all_metrics()

os_helper.write_data(directory_name="transform data", file_name="all_metrics.csv", data_frame=final_df)

Loaded file close_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file volume_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
                  Open        High         Low       Close   Adj Close     Volume
Date                                                                             
1995-01-03   45.703125   45.843750   45.687500   45.781250   27.352005     324300
1995-01-04   45.984375   46.000000   45.750000   46.000000   27.482687     351800
1995-01-05   46.031250   46.109375   45.953125   46.000000   27.482687      89800
1995-01-06   46.093750   46.250000   45.906250   46.046875   27.510689     448400
1995-01-09   46.031250   46.093750   46.000000   46.093750   27.538696      36800
...      

In [ ]:
from get_rebal_dates import GetRebalDates, filter_by_rebalance_dates

all_metrics = os_helper.read_data(directory_name="transform data", file_name="all_metrics.csv", index_col=0, header=[0, 1])
print(all_metrics.head())

termination_date = '2015-12-31' # Last day of the backtest
initial_year = 2000 # First year of the backtest
reb_month = 1
reb_week = 2
reb_weekday = 'FRI'
reb_frequency = 'Q' # Q for quarterly, M for monthly, A for annual, S for semi-annual

# Get the rebalance dates of the strategy
get_reb_dates = GetRebalDates(termination_date=termination_date, initial_year=initial_year, reb_month=reb_month,
                              reb_week=reb_week, reb_weekday=reb_weekday, reb_frequency=reb_frequency)
rebalance_dates = get_reb_dates.reb_dates

print("Dates de rebalancement :")
for date in rebalance_dates:
    print(date)

reb_metrics = filter_by_rebalance_dates(get_reb_dates=get_reb_dates, data=all_metrics)
print(reb_metrics.head())

os_helper.write_data(directory_name="transform data", file_name=f"rebalance_metrics.csv", data_frame=reb_metrics)

In [ ]:
from filter import Filter

data = os_helper.read_data(directory_name="transform data", file_name="rebalance_metrics.csv", index_col=0)
print(data)
risk_filter = Filter(data=data)

risk_top_n = 120
liquidity_top_n = 200

filtered_data = risk_filter.apply_filters(liquidity_top_n=liquidity_top_n, risk_top_n=risk_top_n)
print(filtered_data.head())

os_helper.write_data(directory_name="transform data", file_name="filtered_data.csv", data_frame=filtered_data)

In [ ]:
from weighting import InverseMetricsWeighting

filtered_data = os_helper.read_data(directory_name="transform data", file_name="filtered_data.csv", index_col=0)
print(filtered_data)

years_to_inverse = [1, 3]
metric_to_inverse = "beta"
weighting_list = [0.5, 0.5]
min_weight = 0.00001
max_weight = 0.05

inverse_metrics_weighting = InverseMetricsWeighting(filtered_data=filtered_data, years_list=years_to_inverse,
                                                    target_metric=metric_to_inverse, weight_list=weighting_list,
                                                    min_limit=min_weight, max_limit=max_weight)
inverse_metrics_weighting_df = inverse_metrics_weighting.compute_inverse_weighted_metrics()
print(inverse_metrics_weighting_df)

os_helper.write_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", data_frame=inverse_metrics_weighting_df)

In [ ]:
from portfolio_returns import PortfolioReturns

prices = os_helper.read_data(directory_name="base data", file_name="tot_retl.csv", index_col=0, sep=',')
print(prices.head())
weighting_df = os_helper.read_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", index_col=0)
print(weighting_df.head())

rebalance_lag = 7 # delay between weight computation and implementation
transaction_fee = 0.001 # 0.1% transaction fee

pf_returns = PortfolioReturns(prices=prices, weighting_data=weighting_df, rebalance_lag=rebalance_lag, transaction_fee=transaction_fee)
drifted_weights = pf_returns.drifted_weights
print(drifted_weights.head())
portfolio_returns = pf_returns.portfolio_returns
print(portfolio_returns.head())

os_helper.write_data(directory_name="final data", file_name="portfolio_returns.csv", data_frame=portfolio_returns)
os_helper.write_data(directory_name="final data", file_name="drifted_weights.csv", data_frame=drifted_weights)

In [ ]:
from perf_report import PerfReport

benchmark_prices = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0)
benchmark_prices.sort_index(inplace=True)
benchmark_prices.index.name = None
benchmark_prices = benchmark_prices[["Adj Close"]]
benchmark_prices.columns = ["SPY"]

print(benchmark_prices.head())

strategy_returns = os_helper.read_data(directory_name="final data", file_name="portfolio_returns.csv", index_col=0)
print(strategy_returns.head())
strategy_returns.columns = ["Portfolio_Returns"]

perf_report = PerfReport(port_returns=strategy_returns, bench_returns=benchmark_prices, strat_name="InverseMetrics")
print(perf_report.get_performance_indicators(rf=0.01, periods_per_year=252))
perf_report.generate_html_report(rf=0.01, periods_per_year=252)